In [1]:
import multiprocessing
import os
import sys
import time
sys.path.append("..")
from sklearn.model_selection import train_test_split

from movingpose.estimator import neighbors
from movingpose.estimator import classifiers

from movingpose.preprocessing import moving_pose
from movingpose.preprocessing import kinect_skeleton_data

## Pickle multiview action data

In [2]:
kinect_skeleton_data.pickle_dir("../pickle/multiview.p", "../ext/dataset/multiview_action/")

Pickling skeleton data...
Saving data...
Saved data to file: ../pickle/multiview.p
Program took 17.773548126220703 to run!


## Load pickled multiview action data

In [3]:
raw_data_dict = kinect_skeleton_data.load_pickle("../pickle/multiview.p")

## Format multiview action data

In [4]:
X, labels = moving_pose.format_skeleton_data_dict(raw_data_dict)

## Perform basic verification that skeleton data is formatted correctly

In [5]:
for action in X:
    for frame_num, pose in enumerate(action):
        assert pose.shape == (20, 10), f"{pose.shape} =/= (20, 10)"
        for i, descriptor in enumerate(pose):
            if i == 0:
                assert list(descriptor)[:-1] == [0, 0, 0, 0, 0, 0, 0, 0, 0]
            assert descriptor[-1] == frame_num, f"{descriptor[-1]} =/= {frame_num}"

## Create train/test split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, labels, random_state=42)

## Pickle Action Classifiers

In [7]:
def pickle_action_classifiers(n_training_neighbors, cache_path):
    nearest_pose_estimator = neighbors.NearestPoses(
        n_neighbors=20,
        n_training_neighbors=n_training_neighbors,
        alpha=0.75,
        beta=0.6,
        kappa=80
    )
    action_classifier = classifiers.ActionClassifier(
        nearest_pose_estimator=nearest_pose_estimator,
        theta=0.5,
        n=80
    )
    action_classifier.fit(
        X_train,
        y_train,
        cache_path=cache_path,
        verbose=True
    )

cache_workers = []
for n_training_neighbors in [2000, 5000, 10000, 20000, 30000, 40000]:
    cache_path = f"../pickle/action_classifier_training-{str(n_training_neighbors)}.p"
    if not os.path.exists(cache_path):
        worker = multiprocessing.Process(
            target=
                pickle_action_classifiers,
            args=(
                n_training_neighbors,
                cache_path
            )
        )
        cache_workers.append(worker)

for cache_worker in cache_workers:
    cache_worker.start()
    
for cache_worker in cache_workers:
    cache_worker.join()

## Train Action Classifier with normalized training data

In [8]:
def score(return_dict, n_neighbors, n_training_neighbors, alpha, beta, kappa, theta, n):
    nearest_pose_estimator = neighbors.NearestPoses(
        n_neighbors=n_neighbors,
        n_training_neighbors=n_training_neighbors,
        alpha=alpha,
        beta=beta,
        kappa=kappa
    )
    action_classifier = classifiers.ActionClassifier(
        nearest_pose_estimator=nearest_pose_estimator,
        theta=theta,
        n=n
    )
    action_classifier.fit(
        X_train,
        y_train,
        cache_path=f"../pickle/action_classifier_training-{str(n_training_neighbors)}.p"
    )
    result = action_classifier.predict_all(X_test)
    return_dict[(n_neighbors, n_training_neighbors, alpha, beta, kappa, theta, n)] = (action_classifier, result)

manager = multiprocessing.Manager()
return_dict = manager.dict()
workers = []

for n_neighbors in [30, 40, 50]:
    for n_training_neighbors in [2000, 5000, 20000, 40000]:
        for alpha in [0.75]:
            for beta in [0.6]:
                for kappa in [30, 40, 50]:
                    for theta in [0.6, 0.7, 0.8, 0.9]:
                        for n in [40, 50, 60]:
                            worker = multiprocessing.Process(
                                target=
                                    score,
                                args=(
                                    return_dict,
                                    n_neighbors,
                                    n_training_neighbors,
                                    alpha,
                                    beta,
                                    kappa,
                                    theta,
                                    n
                                )
                            )
                            workers.append(worker)

num_cpu_cores = os.getenv("NUM_CPU_CORES")
if num_cpu_cores is None:
    num_cpu_cores = input("Enter the number of CPU cores this computer has (ex. '12'): ")
num_cpu_cores = max(1, int(num_cpu_cores) - 4)

timer = time.time()
num_workers_executed = num_cpu_cores
while num_workers_executed < len(workers):
    for i in range(num_cpu_cores):
        workers[num_workers_executed - i].start()
    for i in range(num_cpu_cores):
        os.system('clear')
        print(f"Tuning {100*round((num_workers_executed-(num_cpu_cores - i))/len(workers), 3)}% complete")
        print(f"Estimated runtime info: {timer}/{timer/((num_workers_executed-(num_cpu_cores + i))/len(workers))} seconds")
        workers[num_workers_executed - i].join()
    num_workers_executed += num_cpu_cores


print("# Correctly predicted: (higher is better)")
best_score = sys.maxsize
best_hpc = None
best_action_classifier = None
for hyper_parameter_combo, (action_classifier, result) in return_dict.items():
    if result > best_score:
        best_hpc = hyper_parameter_combo
        best_action_classifier = action_classifier
        best_score = result


Training 0.0% complete
Training -0.2% complete
Training -0.5% complete
Training -0.7000000000000001% complete
Training -0.8999999999999999% complete
Training -1.2% complete
Training -1.4000000000000001% complete
Training -1.6% complete
Training -1.9% complete
Training -2.1% complete
Training -2.3% complete
Training -2.5% complete
Training -2.8000000000000003% complete
Training -3.0% complete
Training -3.2% complete
Training -3.5000000000000004% complete
Training -3.6999999999999997% complete
Training -3.9% complete
Training -4.2% complete
Training -4.3999999999999995% complete
Training 4.6% complete


Process Process-11:
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-8-da8517d8986f>", line 19, in score
    result = action_classifier.predict_all(X_test)
  File "/home/mobley/Documents/repos/CSCI470-Final/movingpose/estimator/classifiers.py", line 123, in predict_all
    result.append(self.predict(Xs[i], poses_are_normalized, verbose=verbose))
  File "/home/mobley/Documents/repos/CSCI470-Final/movingpose/estimator/classifiers.py", line 95, in predict
    for nearby_pose, score in self.nearest_pose_estimator.k_poses(pose, verbose=verbose):
  File "/home/mobley/Documents/repos/CSCI470-Final/movingpose/estimator/neighbors.py", line 206, in k_poses
    traditional_knn = KNeighborsClassifier(
  File "/home/mobley/Documents/repos/CSCI470-Final/venv/lib/python3.8/site-pac

KeyboardInterrupt: 

## Print best action classifier and its score

In [ ]:
print("The best hyper parameter combo was:")
print(f"    n_neighbors: {best_hpc[0]}")
print(f"    n_training_neighbors: {best_hpc[1]}")
print(f"    alpha: {best_hpc[2]}")
print(f"    beta: {best_hpc[3]}")
print(f"    kappa: {best_hpc[4]}")
print(f"    theta: {best_hpc[5]}")
print(f"    n: {best_hpc[6]}")
print("\n--------------\n")
print(f"Its score was: {best_score}")

## Save the best action classifier

In [ ]:
best_action_classifier.save_pickle("../pickle/best_action_classifier.p")

## Load best action classifier

In [ ]:
action_classifier = classifiers.load_pickle("../pickle/best_action_classifier.p")

## Save the best action classifier

In [ ]:
best_action_classifier.save_pickle("../pickle/best_action_classifier.p")

## Load best action classifier

In [ ]:
action_classifier = classifiers.load_pickle("../pickle/best_action_classifier.p")

## Predict live data

In [ ]:
#TODO add live predictions when mehmet publishes the UI code

# DAB

In [ ]:
print("dab")
